# 구글비전api 테스트

In [1]:
# from google.cloud import vision
# import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspace/text2palette/profound-actor-347102-b0e7c4a8632c.json"

# def detect_labels(image_content):   
#     client = vision.ImageAnnotatorClient()
#     image = vision.Image(content=image_content)
#     response = client.label_detection(image=image, max_results=30)
#     labels = response.label_annotations
    
#     if response.error.message:
#         raise Exception(response.error.message)
    
#     return labels

# image_path = "/data/shared/xml_image_csv_converter/image/group_false/241114/24111400/31005/241112_31005_0.png"

# with open(image_path, 'rb') as img_file:
#     image_content = img_file.read()

# labels = detect_labels(image_content)
# for label in labels:
#     print(label.description, label.score)

# 데이터 불러오기

In [ ]:
import pandas as pd

csv_path = "/data/shared/xml_image_csv_converter/csv/group_true"
date = ["250311", "250312_250326", "250327", "250328", "250329", "250330", "250331", "250401", "250402"] # 241121 포맷이 달라서 잠시 제외

df_list = [pd.read_csv(f"{csv_path}/{d}/{d}.csv") for d in date]

combined_df = pd.concat(df_list, ignore_index=True)

In [3]:
combined_df.head()

,template_idx,page_num,background,image_file_name,img_width_resized,img_height_resized,tag,is_text,text_content,tbpeID,...,img_height,priority,rotation,opacity,vertical_align,text_align,font_size,font_type,color,group_id
0,709559,0,True,709559_0_Thumbnail.png,1920,1080,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,...,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,Thumbnail_Images,NaN
1,709559,0,True,709559_0_Skin.png,1920,1080,Skin_Images,Skin_Images,Skin_Images,Skin_Images,...,Skin_Images,Skin_Images,Skin_Images,Skin_Images,Skin_Images,Skin_Images,Skin_Images,Skin_Images,Skin_Images,NaN
2,709559,0,False,709559_0_0.png,0,0,GENERALSVG,FALSE,Not_Exists,1739872082453-0.5312638551548066,...,1090.5068142766354,0,0.0,255.0,Not_Exists,Not_Exists,Not_Exists,Not_Exists,#f8ead0,NaN
3,709559,0,False,709559_0_1.png,0,0,PHOTO,FALSE,Not_Exists,1739872082451-0.5200572357864228,...,799.6908117931614,1,0.0,255.0,Not_Exists,Not_Exists,Not_Exists,Not_Exists,Not_Exist,NaN
4,709559,0,False,709559_0_2.png,0,0,SHAPESVG,FALSE,Not_Exists,1739872082453-0.07207545610789778,...,752.612021857924,2,0.0,255.0,Not_Exists,Not_Exists,Not_Exists,Not_Exists,none,NaN


In [4]:
len(combined_df)

100588

In [5]:
import pandas as pd
import ast

# group_key 생성
combined_df['group_key'] = combined_df['template_idx'].astype(str) + '_' + combined_df['page_num'].astype(str)

# Thumbnail_Images와 Skin_Images 제거
combined_df = combined_df[
    ~combined_df['tag'].astype(str).isin(['Thumbnail_Images', 'Skin_Images'])
]

# LineShapeItem 제거
combined_df = combined_df[
    ~combined_df['tag'].astype(str).str.contains('LineShapeItem')
]

# simplified_tag 판단 함수
def determine_simplified_tag(row):
    tag = row['tag']
    color = row['color']
    try:
        tag_list = ast.literal_eval(tag) if isinstance(tag, str) and tag.startswith("[") else [tag]
    except:
        tag_list = [tag]

    tag_list = [str(t) for t in tag_list]

    if any('TEXT' in t for t in tag_list):
        return 'TEXT'
    if 'FrameItem' in tag_list and 'PHOTO' in tag_list:
        return 'PHOTO'
    if 'FrameItem' in tag_list and 'SVG' in tag_list:
        return 'SVG'
    if 'PHOTO' in tag_list:
        return 'PHOTO'
    if 'FrameItem' not in tag_list and color != 'Not_Exist':
        return 'SVG'
    return None

combined_df['simplified_tag'] = combined_df.apply(determine_simplified_tag, axis=1)

# color 컬럼 파싱
def parse_color(c):
    try:
        if isinstance(c, str) and c.startswith("["):
            return ast.literal_eval(c)
        else:
            return [c]
    except:
        return [c]

combined_df['parsed_color'] = combined_df['color'].apply(parse_color)

# text_content 파싱
def parse_text_content(t):
    try:
        if isinstance(t, str) and t.startswith("["):
            return ast.literal_eval(t)
        else:
            return [t]
    except:
        return [t]

combined_df['parsed_text'] = combined_df['text_content'].apply(parse_text_content)

# explode는 먼저 parsed_color만 적용
color_exploded_df = combined_df[combined_df['simplified_tag'].notna()].explode('parsed_color')

# 그 다음 text_content는 리스트든 문자열이든 관계없이 groupby에서 그대로 리스트로 모으기
palette_df = (
    color_exploded_df
    .groupby(['group_key', 'simplified_tag'])
    .agg({
        'parsed_color': list,
        'text_content': lambda x: list(x),  # 원본 텍스트 모으기
        'image_file_name': list             # 이미지 파일명도 리스트로 저장
    })
    .reset_index()
    .rename(columns={
        'parsed_color': 'palette',
        'text_content': 'text_contents'
    })
)

palette_df.head(10)


,group_key,simplified_tag,palette,text_contents,image_file_name
0,709559_0,PHOTO,"[Not_Exist, Not_Exist, Not_Exist, Not_Exist, N...","[Not_Exists, Not_Exists, Not_Exists, Not_Exist...","[709559_0_1.png, 709559_0_6.png, 709559_0_8.pn..."
1,709559_0,SVG,"[#f8ead0, none, #ffffff, #319a44, #0a7b30, #0a...","[Not_Exists, Not_Exists, Not_Exists, Not_Exist...","[709559_0_0.png, 709559_0_2.png, 709559_0_3.pn..."
2,709559_0,TEXT,"[#319a44, #000000, #353535, #353535, #ffffff]","[미리 유치원, 제 0회 졸업식, ['어린이 여러분의 ', '졸업을 축하합니다'],...","[709559_0_28.png, 709559_0_29.png, 709559_0_30..."
3,709559_1,PHOTO,"[Not_Exist, Not_Exist, Not_Exist]","[Not_Exists, Not_Exists, Not_Exists]","[709559_1_1.png, 709559_1_5.png, 709559_1_6.png]"
4,709559_1,SVG,"[#f8ead0, #319a44, none, #ffffff, #ffc23b, #ff...","[Not_Exists, Not_Exists, Not_Exists, Not_Exist...","[709559_1_0.png, 709559_1_2.png, 709559_1_3.pn..."
5,709559_1,TEXT,"[#252525, #353535, #ffffff, #353535, #FFFFFFFF...","[식 순, 애국가 제창, 01, 원장님 말씀, 02, 졸업장 수여, 03, 선생님...","[709559_1_11.png, 709559_1_13.png, 709559_1_14..."
6,709559_2,PHOTO,"[Not_Exist, Not_Exist, Not_Exist, Not_Exist, N...","[Not_Exists, Not_Exists, Not_Exists, Not_Exist...","[709559_2_7.png, 709559_2_12.png, 709559_2_13...."
7,709559_2,SVG,"[#f9f8f8, #72b257, none, #ffffff, #0a7b30, #31...","[Not_Exists, Not_Exists, Not_Exists, Not_Exist...","[709559_2_0.png, 709559_2_1.png, 709559_2_2.pn..."
8,709559_2,TEXT,"[#ffffff, #353535, #FFFFFFFF, #353535, #353535...","[1절, 동해물과 백두산이 마르고 닳도록, 하느님이 보우하사 우리 나라 만 세, 2...","[709559_2_28.png, 709559_2_29.png, 709559_2_30..."
9,709559_3,PHOTO,"[Not_Exist, #040000, #a24c4c, Not_Exist, Not_E...","[Not_Exists, ['Not_Exists', 'Not_Exists'], ['N...","[709559_3_5.png, 709559_3_7.png, 709559_3_7.pn..."


In [6]:
len(palette_df)

10868

# CSV파일로 정리

In [7]:
import pandas as pd
import ast
import sys
sys.path.append('../src')

from color_palette_completion.utils.color_convertor import rgb_to_lab

# palette 컬럼에서 색상 추출
def hex_to_rgb(hex_color):
    if hex_color in ['Not_Exist', 'none', None] or not isinstance(hex_color, str):
        return None
    hex_color = hex_color.lstrip('#')
    try:
        return [int(hex_color[i:i+2], 16) for i in (0, 2, 4)]
    except:
        return None

def convert_palette(palette_list):
    rgb_colors = [hex_to_rgb(c) for c in palette_list]
    rgb_colors = [c for c in rgb_colors if c is not None]
    return rgb_colors

def convert_to_lab(rgb_list):
    return [rgb_to_lab(c) for c in rgb_list]

def reorder_lab(lab_colors):
    return sorted(lab_colors, key=lambda x: x[0])  # L 값 기준 정렬

# 그룹별 처리
output_data = []

for group_key, group_df in palette_df.groupby('group_key'):
    row = {'group_key': group_key}
    
    # 모든 text_contents를 통합 (중복 제거)
    all_texts = group_df['text_contents'].explode().dropna().unique().tolist()
    row['text_contents'] = all_texts

    # TEXT가 아닌 image_file_name만 수집
    image_file_names = (
        group_df[group_df['simplified_tag'] != 'TEXT']['image_file_name']
        .explode()
        .dropna()
        .unique()
        .tolist()
    )
    row['image_file_name'] = image_file_names

    for tag in ['PHOTO', 'SVG', 'TEXT']:
        subset = group_df[group_df['simplified_tag'] == tag]
        if not subset.empty:
            palette = subset.iloc[0]['palette']
            rgb_colors = convert_palette(palette)
            lab_colors = convert_to_lab(rgb_colors)
            lab_colors_reorder = reorder_lab(lab_colors)
        else:
            rgb_colors = lab_colors = lab_colors_reorder = []

        row[f'{tag.lower()}_colors'] = rgb_colors
        row[f'{tag.lower()}_colors_lab'] = lab_colors
        row[f'{tag.lower()}_colors_lab_reorder'] = lab_colors_reorder

    output_data.append(row)

# 최종 DataFrame
final_df = pd.DataFrame(output_data)

# 컬럼 순서 정리
ordered_columns = [
    'group_key', 'text_contents', 'image_file_name',
    'photo_colors', 'svg_colors', 'text_colors',
    'photo_colors_lab', 'svg_colors_lab', 'text_colors_lab',
    'photo_colors_lab_reorder', 'svg_colors_lab_reorder', 'text_colors_lab_reorder',
]
final_df = final_df[ordered_columns]

# 저장
# final_df.to_csv('text_palette_extracted.csv', index=False)


In [8]:
final_df.head(10)

,group_key,text_contents,image_file_name,photo_colors,svg_colors,text_colors,photo_colors_lab,svg_colors_lab,text_colors_lab,photo_colors_lab_reorder,svg_colors_lab_reorder,text_colors_lab_reorder
0,709559_0,"[Not_Exists, ['Not_Exists', 'Not_Exists'], 미리 ...","[709559_0_1.png, 709559_0_6.png, 709559_0_8.pn...",[],"[[248, 234, 208], [255, 255, 255], [49, 154, 6...","[[49, 154, 68], [0, 0, 0], [53, 53, 53], [53, ...",[],"[[238, 129, 142], [255, 128, 128], [143, 80, 1...","[[143, 80, 164], [0, 128, 128], [57, 128, 128]...",[],"[[57, 128, 128], [57, 128, 128], [90, 89, 157]...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
1,709559_1,"[Not_Exists, 식 순, 애국가 제창, 01, 원장님 말씀, 02, 졸업장...","[709559_1_1.png, 709559_1_5.png, 709559_1_6.pn...",[],"[[248, 234, 208], [49, 154, 68], [255, 255, 25...","[[37, 37, 37], [53, 53, 53], [255, 255, 255], ...",[],"[[238, 129, 142], [143, 80, 164], [255, 128, 1...","[[37, 128, 128], [57, 128, 128], [255, 128, 12...",[],"[[0, 128, 128], [57, 128, 128], [90, 89, 157],...","[[37, 128, 128], [57, 128, 128], [57, 128, 128..."
2,709559_2,"[Not_Exists, 1절, 동해물과 백두산이 마르고 닳도록, 하느님이 보우하사 ...","[709559_2_7.png, 709559_2_12.png, 709559_2_13....",[],"[[249, 248, 248], [114, 178, 87], [255, 255, 2...","[[255, 255, 255], [53, 53, 53], [255, 255, 255...",[],"[[249, 128, 128], [170, 91, 168], [255, 128, 1...","[[255, 128, 128], [57, 128, 128], [255, 128, 1...",[],"[[0, 128, 128], [53, 141, 88], [57, 128, 128],...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
3,709559_3,"[Not_Exists, ['Not_Exists', 'Not_Exists'], 원장님...","[709559_3_5.png, 709559_3_7.png, 709559_3_10.p...","[[4, 0, 0], [162, 76, 76]]","[[249, 248, 248], [114, 178, 87], [49, 154, 68...","[[0, 0, 0], [199, 199, 199], [119, 119, 119], ...","[[1, 129, 128], [110, 164, 145]]","[[249, 128, 128], [170, 91, 168], [143, 80, 16...","[[0, 128, 128], [205, 128, 128], [128, 128, 12...","[[1, 129, 128], [110, 164, 145]]","[[0, 128, 128], [57, 128, 128], [114, 82, 160]...","[[0, 128, 128], [128, 128, 128], [128, 128, 12..."
4,709559_4,"[['Not_Exists', 'Not_Exists'], Not_Exists, ['위...","[709559_4_4.png, 709559_4_12.png, 709559_4_23....","[[160, 160, 160]]","[[249, 248, 248], [114, 178, 87], [255, 255, 2...","[[53, 53, 53], [53, 53, 53], [53, 53, 53], [53...","[[168, 128, 128]]","[[249, 128, 128], [170, 91, 168], [255, 128, 1...","[[57, 128, 128], [57, 128, 128], [57, 128, 128...","[[168, 128, 128]]","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
5,709559_5,"[Not_Exists, 선생님의 배웅, 사랑반, 윤미리선생님, ['미리 유치원의\x...","[709559_5_5.png, 709559_5_0.png, 709559_5_1.pn...",[],"[[249, 248, 248], [114, 178, 87], [255, 255, 2...","[[0, 0, 0], [255, 255, 255], [255, 255, 255], ...",[],"[[249, 128, 128], [170, 91, 168], [255, 128, 1...","[[0, 128, 128], [255, 128, 128], [255, 128, 12...",[],"[[114, 82, 160], [114, 82, 160], [114, 82, 160...","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ..."
6,709559_6,"[Not_Exists, ['Not_Exists', 'Not_Exists'], 우리의...","[709559_6_1.png, 709559_6_5.png, 709559_6_6.pn...","[[255, 255, 255], [0, 0, 0], [255, 255, 255], ...","[[248, 234, 208], [114, 178, 87], [255, 255, 2...","[[0, 0, 0], [53, 53, 53], [255, 255, 255], [53...","[[255, 128, 128], [0, 128, 128], [255, 128, 12...","[[238, 129, 142], [170, 91, 168], [255, 128, 1...","[[0, 128, 128], [57, 128, 128], [255, 128, 128...","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ...","[[114, 82, 160], [114, 82, 160], [143, 80, 164...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
7,709559_7,"[Not_Exists, 졸업을 축하합니다!, 즐겁고 활기찬 초등학교 생활 되길 바라요]","[709559_7_1.png, 709559_7_9.png, 709559_7_10.p...",[],"[[248, 234, 208], [114, 178, 87], [10, 123, 48...","[[49, 154, 68], [53, 53, 53]]",[],"[[238, 129, 142], [170, 91, 168], [114, 82, 16...","[[143, 80, 164], [57, 128, 128]]",[],"[[57, 128, 128], [57, 128, 128], [114, 82, 160...","[[57, 128, 128], [143, 80, 164]]"
8,709560_0,"[Not_Exists, ['실무에서 꼭 필요한', ' ', '기본 구성 보고서'],...","[709560_0_0.png, 70

In [9]:
len(final_df)

4255

In [10]:
def is_valid_palette(palette):
    return isinstance(palette, list) and len(palette) > 0

# 조건에 맞는 행만 필터링
preprocess_final_df = final_df[
    final_df['photo_colors'].apply(is_valid_palette) &
    final_df['svg_colors'].apply(is_valid_palette) &
    final_df['text_colors'].apply(is_valid_palette)
].reset_index(drop=True)

In [11]:
preprocess_final_df.head(10)

,group_key,text_contents,image_file_name,photo_colors,svg_colors,text_colors,photo_colors_lab,svg_colors_lab,text_colors_lab,photo_colors_lab_reorder,svg_colors_lab_reorder,text_colors_lab_reorder
0,709559_3,"[Not_Exists, ['Not_Exists', 'Not_Exists'], 원장님...","[709559_3_5.png, 709559_3_7.png, 709559_3_10.p...","[[4, 0, 0], [162, 76, 76]]","[[249, 248, 248], [114, 178, 87], [49, 154, 68...","[[0, 0, 0], [199, 199, 199], [119, 119, 119], ...","[[1, 129, 128], [110, 164, 145]]","[[249, 128, 128], [170, 91, 168], [143, 80, 16...","[[0, 128, 128], [205, 128, 128], [128, 128, 12...","[[1, 129, 128], [110, 164, 145]]","[[0, 128, 128], [57, 128, 128], [114, 82, 160]...","[[0, 128, 128], [128, 128, 128], [128, 128, 12..."
1,709559_4,"[['Not_Exists', 'Not_Exists'], Not_Exists, ['위...","[709559_4_4.png, 709559_4_12.png, 709559_4_23....","[[160, 160, 160]]","[[249, 248, 248], [114, 178, 87], [255, 255, 2...","[[53, 53, 53], [53, 53, 53], [53, 53, 53], [53...","[[168, 128, 128]]","[[249, 128, 128], [170, 91, 168], [255, 128, 1...","[[57, 128, 128], [57, 128, 128], [57, 128, 128...","[[168, 128, 128]]","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
2,709559_6,"[Not_Exists, ['Not_Exists', 'Not_Exists'], 우리의...","[709559_6_1.png, 709559_6_5.png, 709559_6_6.pn...","[[255, 255, 255], [0, 0, 0], [255, 255, 255], ...","[[248, 234, 208], [114, 178, 87], [255, 255, 2...","[[0, 0, 0], [53, 53, 53], [255, 255, 255], [53...","[[255, 128, 128], [0, 128, 128], [255, 128, 12...","[[238, 129, 142], [170, 91, 168], [255, 128, 1...","[[0, 128, 128], [57, 128, 128], [255, 128, 128...","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ...","[[114, 82, 160], [114, 82, 160], [143, 80, 164...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
3,709560_3,"[['Not_Exists', 'Not_Exists'], ['Not_Exists', ...","[709560_3_3.png, 709560_3_0.png, 709560_3_1.pn...","[[145, 70, 70]]","[[50, 147, 252], [175, 214, 255], [255, 255, 9...","[[0, 0, 0], [0, 0, 0], [89, 89, 89], [85, 85, ...","[[100, 160, 143]]","[[154, 137, 68], [215, 125, 104], [249, 109, 2...","[[0, 128, 128], [0, 128, 128], [96, 128, 128],...","[[100, 160, 143]]","[[0, 128, 128], [22, 128, 128], [154, 137, 68]...","[[0, 128, 128], [0, 128, 128], [37, 128, 128],..."
4,709560_4,"[['Not_Exists', 'Not_Exists'], ['Not_Exists', ...","[709560_4_3.png, 709560_4_4.png, 709560_4_5.pn...","[[160, 160, 160], [160, 160, 160], [160, 160, ...","[[50, 147, 252], [175, 214, 255], [255, 255, 9...","[[0, 0, 0], [85, 85, 85], [0, 0, 0], [89, 89, ...","[[168, 128, 128], [168, 128, 128], [168, 128, ...","[[154, 137, 68], [215, 125, 104], [249, 109, 2...","[[0, 128, 128], [92, 128, 128], [0, 128, 128],...","[[168, 128, 128], [168, 128, 128], [168, 128, ...","[[0, 128, 128], [154, 137, 68], [154, 137, 68]...","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ..."
5,709568_0,"[Not_Exists, ['Not_Exists', 'Not_Exists'], ['N...","[709568_0_0.png, 709568_0_1.png, 709568_0_2.pn...","[[160, 160, 160]]","[[255, 255, 255], [255, 255, 255], [255, 255, ...","[[255, 234, 153], [255, 234, 153], [255, 234, ...","[[168, 128, 128]]","[[255, 128, 128], [255, 128, 128], [255, 128, ...","[[237, 124, 170], [237, 124, 170], [237, 124, ...","[[168, 128, 128]]","[[0, 128, 128], [96, 132, 96], [96, 132, 96], ...","[[96, 132, 96], [96, 132, 96], [96, 132, 96], ..."
6,709574_2,"[['Not_Exists', 'Not_Exists'], Not_Exists, 당신의...","[709574_2_7.png, 709574_2_9.png, 709574_2_0.pn...","[[89, 89, 193]]","[[255, 237, 136], [0, 0, 0], [255, 255, 255], ...","[[255, 255, 255], [255, 255, 255]]","[[110, 157, 73]]","[[238, 121, 179], [0, 128, 128], [255, 128, 12...","[[255, 128, 128], [255, 128, 128]]","[[110, 157, 73]]","[[0, 128, 128], [0, 128, 128], [131, 183, 178]...","[[255, 128, 128], [255, 128, 128]]"
7,709575_1,"[['Not_Exists', 'Not_Exists'], Not_Exists, 귀여운...","[709575_1_5.png, 709575_1_10.png, 709575_1_22....","[[30, 30, 30], [202, 202, 202], [30, 30, 30], ...","[[242, 110, 2], [0, 0, 0], [255, 255, 255], [2

In [ ]:
import ast

def clean_text_contents(entry):
    cleaned = []

    for item in entry:
        # 문자열인데 리스트처럼 생긴 경우 (예: "['Not_Exists', 'Not_Exists']")
        if isinstance(item, str) and item.startswith('[') and item.endswith(']'):
            try:
                parsed = ast.literal_eval(item)
                if isinstance(parsed, list):
                    # 리스트 내 모든 값이 'Not_Exists'이면 제거
                    valid = [sub for sub in parsed if sub != 'Not_Exists']
                    cleaned.extend(valid)
            except:
                # 파싱 실패 시 무시
                continue

        elif isinstance(item, str):
            if item != 'Not_Exists':
                cleaned.append(item)

        elif isinstance(item, list):
            valid = [sub for sub in item if sub != 'Not_Exists']
            cleaned.extend(valid)

    return cleaned

# 적용
preprocess_final_df['text_contents'] = preprocess_final_df['text_contents'].apply(clean_text_contents)
preprocess_final_df = preprocess_final_df[preprocess_final_df['text_contents'].map(len) > 0].reset_index(drop=True)
preprocess_final_df.head(10)

,group_key,text_contents,image_file_name,photo_colors,svg_colors,text_colors,photo_colors_lab,svg_colors_lab,text_colors_lab,photo_colors_lab_reorder,svg_colors_lab_reorder,text_colors_lab_reorder
0,709559_3,"[원장님 말씀, *페이지 내 인물 사진은 샘플이미지 입니다., 해당 순서에 대한 설...","[709559_3_5.png, 709559_3_7.png, 709559_3_10.p...","[[4, 0, 0], [162, 76, 76]]","[[249, 248, 248], [114, 178, 87], [49, 154, 68...","[[0, 0, 0], [199, 199, 199], [119, 119, 119], ...","[[1, 129, 128], [110, 164, 145]]","[[249, 128, 128], [170, 91, 168], [143, 80, 16...","[[0, 128, 128], [205, 128, 128], [128, 128, 12...","[[1, 129, 128], [110, 164, 145]]","[[0, 128, 128], [57, 128, 128], [114, 82, 160]...","[[0, 128, 128], [128, 128, 128], [128, 128, 12..."
1,709559_4,"[위 어린이는 본 원에서 모든 과정을, 수료하였기에, 졸업장을 수여합니다., 209...","[709559_4_4.png, 709559_4_12.png, 709559_4_23....","[[160, 160, 160]]","[[249, 248, 248], [114, 178, 87], [255, 255, 2...","[[53, 53, 53], [53, 53, 53], [53, 53, 53], [53...","[[168, 128, 128]]","[[249, 128, 128], [170, 91, 168], [255, 128, 1...","[[57, 128, 128], [57, 128, 128], [57, 128, 128...","[[168, 128, 128]]","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
2,709559_6,"[우리의 추억, 2089.03.15, 형님반 첫 등반, 2089.05.03, 어린이...","[709559_6_1.png, 709559_6_5.png, 709559_6_6.pn...","[[255, 255, 255], [0, 0, 0], [255, 255, 255], ...","[[248, 234, 208], [114, 178, 87], [255, 255, 2...","[[0, 0, 0], [53, 53, 53], [255, 255, 255], [53...","[[255, 128, 128], [0, 128, 128], [255, 128, 12...","[[238, 129, 142], [170, 91, 168], [255, 128, 1...","[[0, 128, 128], [57, 128, 128], [255, 128, 128...","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ...","[[114, 82, 160], [114, 82, 160], [143, 80, 164...","[[0, 128, 128], [57, 128, 128], [57, 128, 128]..."
3,709560_3,"[MIRI COMPANY, 주제를 입력하세요., 이곳에 왼쪽 사진에 관한 정보를 입...","[709560_3_3.png, 709560_3_0.png, 709560_3_1.pn...","[[145, 70, 70]]","[[50, 147, 252], [175, 214, 255], [255, 255, 9...","[[0, 0, 0], [0, 0, 0], [89, 89, 89], [85, 85, ...","[[100, 160, 143]]","[[154, 137, 68], [215, 125, 104], [249, 109, 2...","[[0, 128, 128], [0, 128, 128], [96, 128, 128],...","[[100, 160, 143]]","[[0, 128, 128], [22, 128, 128], [154, 137, 68]...","[[0, 128, 128], [0, 128, 128], [37, 128, 128],..."
4,709560_4,"[MIRI COMPANY, 사진과 함께 세개의 단으로 나눠 설명하기 좋은 페이지입...","[709560_4_3.png, 709560_4_4.png, 709560_4_5.pn...","[[160, 160, 160], [160, 160, 160], [160, 160, ...","[[50, 147, 252], [175, 214, 255], [255, 255, 9...","[[0, 0, 0], [85, 85, 85], [0, 0, 0], [89, 89, ...","[[168, 128, 128], [168, 128, 128], [168, 128, ...","[[154, 137, 68], [215, 125, 104], [249, 109, 2...","[[0, 128, 128], [92, 128, 128], [0, 128, 128],...","[[168, 128, 128], [168, 128, 128], [168, 128, ...","[[0, 128, 128], [154, 137, 68], [154, 137, 68]...","[[0, 128, 128], [0, 128, 128], [0, 128, 128], ..."
5,709568_0,"[ㅌ, ㅍ, ㅇ, 퀴즈 이벤트, 초성 맞추기, 정답 맞추고 여행가자!, 댓글로 정답...","[709568_0_0.png, 709568_0_1.png, 709568_0_2.pn...","[[160, 160, 160]]","[[255, 255, 255], [255, 255, 255], [255, 255, ...","[[255, 234, 153], [255, 234, 153], [255, 234, ...","[[168, 128, 128]]","[[255, 128, 128], [255, 128, 128], [255, 128, ...","[[237, 124, 170], [237, 124, 170], [237, 124, ...","[[168, 128, 128]]","[[0, 128, 128], [96, 132, 96], [96, 132, 96], ...","[[96, 132, 96], [96, 132, 96], [96, 132, 96], ..."
6,709574_2,"[당신의 보고서가 막히는 이유?, 보고서 마스터 02, 제안...","[709574_2_7.png, 709574_2_9.png, 709574_2_0.pn...","[[89, 89, 193]]","[[255, 237, 136], [0, 0, 0], [255, 255, 255], ...","[[255, 255, 255], [255, 255, 255]]","[[110, 157, 73]]","[[238, 121, 179], [0, 128, 128], [255, 128, 12...","[[255, 128, 128], [255, 128, 128]]","[[110, 157, 73]]","[[0, 128, 128], [0, 128, 128], [131, 183, 178]...","[[255, 128, 128], [255, 128, 128]]"
7,709575_1,"[귀여운데 실용적이야, 다양한 캐릭터 문구 시리즈, 새학기 기운 팍팍, 아이템 총...","[709575_1_5.png, 709575_1_10.png, 709575_1_22....","[[30, 30, 30], [202, 202, 202], [30, 30, 30], ...","[[242, 110, 2], [0, 0, 0], [255, 255, 255], [2...","[[242, 11

In [16]:
preprocess_final_df['svg_colors'][0]

[[249, 248, 248],
 [114, 178, 87],
 [49, 154, 68],
 [49, 154, 68],
 [49, 154, 68],
 [255, 194, 59],
 [255, 255, 255],
 [10, 123, 48],
 [255, 142, 142],
 [255, 194, 59],
 [10, 123, 48],
 [135, 195, 232],
 [49, 154, 68],
 [10, 123, 48],
 [10, 123, 48],
 [255, 142, 142],
 [255, 255, 255],
 [255, 194, 59],
 [152, 102, 9],
 [255, 142, 142],
 [53, 53, 53],
 [255, 142, 142],
 [0, 0, 0],
 [255, 255, 255],
 [135, 195, 232],
 [135, 195, 232],
 [255, 194, 59],
 [255, 194, 59],
 [10, 123, 48],
 [49, 154, 68],
 [184, 221, 102],
 [255, 255, 255],
 [184, 221, 102],
 [255, 224, 66],
 [10, 123, 48],
 [10, 123, 48],
 [255, 255, 255],
 [255, 223, 153],
 [199, 199, 199]]

In [15]:
len(preprocess_final_df)

1044

In [14]:
preprocess_final_df.to_csv('text_palette_extracted.csv', index=False)